In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras
import warnings
warnings.filterwarnings('ignore')

C:\Users\Owner\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Owner\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Owner\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Owner\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\frame

In [2]:
train_dir = 'C:/dev/dog-breed-identification/train'
test_dir = 'C:/dev/dog-breed-identification/test'

In [3]:
train_size = len(os.listdir('C:/dev/dog-breed-identification/train'))
test_size = len(os.listdir('C:/dev/dog-breed-identification/test'))

train_size,test_size

(10222, 10357)

In [4]:
df = pd.read_csv('C:/dev/dog-breed-identification/labels.csv')
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [5]:
#Extracting different classes
dog_breeds = sorted(df['breed'].unique())
n_classes = len(dog_breeds)
print(n_classes)
dog_breeds

120


['affenpinscher',
 'afghan_hound',
 'african_hunting_dog',
 'airedale',
 'american_staffordshire_terrier',
 'appenzeller',
 'australian_terrier',
 'basenji',
 'basset',
 'beagle',
 'bedlington_terrier',
 'bernese_mountain_dog',
 'black-and-tan_coonhound',
 'blenheim_spaniel',
 'bloodhound',
 'bluetick',
 'border_collie',
 'border_terrier',
 'borzoi',
 'boston_bull',
 'bouvier_des_flandres',
 'boxer',
 'brabancon_griffon',
 'briard',
 'brittany_spaniel',
 'bull_mastiff',
 'cairn',
 'cardigan',
 'chesapeake_bay_retriever',
 'chihuahua',
 'chow',
 'clumber',
 'cocker_spaniel',
 'collie',
 'curly-coated_retriever',
 'dandie_dinmont',
 'dhole',
 'dingo',
 'doberman',
 'english_foxhound',
 'english_setter',
 'english_springer',
 'entlebucher',
 'eskimo_dog',
 'flat-coated_retriever',
 'french_bulldog',
 'german_shepherd',
 'german_short-haired_pointer',
 'giant_schnauzer',
 'golden_retriever',
 'gordon_setter',
 'great_dane',
 'great_pyrenees',
 'greater_swiss_mountain_dog',
 'groenendael',


In [6]:
#Converting classes to numbers
class_to_num = dict(zip(dog_breeds,range(n_classes)))

In [7]:
#Function to load and convert images to array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical

def images_to_array(data_dir,df,image_size):
    image_names = df['id']
    image_labels = df['breed']
    data_size = len(image_names)
    
    X = np.zeros([data_size,image_size[0],image_size[1],image_size[2]],dtype = np.uint8)
    y = np.zeros([data_size,1],dtype = np.uint8)
    
    for i in range(data_size):
        img_name = image_names[i]
        img_dir = os.path.join(data_dir,img_name+'.jpg')
        img_pixels = load_img(img_dir,target_size=image_size)
        X[i] = img_pixels
        y[i] = class_to_num[image_labels[i]]
        
    y = to_categorical(y)
    
    ind = np.random.permutation(data_size)
    X = X[ind]
    y = y[ind]
    print('Ouptut Data Size: ', X.shape)
    print('Ouptut Label Size: ', y.shape)
    return X, y     

Using TensorFlow backend.


In [8]:
#Selecting image size according to pretrained models
img_size = (299,299,3)

In [9]:
X, y = images_to_array(train_dir,df,img_size)

Ouptut Data Size:  (10222, 299, 299, 3)
Ouptut Label Size:  (10222, 120)


In [10]:
#Function to extract features from images
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D,Lambda, Dropout, InputLayer, Input

def get_features(model_name, data_preprocessor, input_size, data):
    #Prepare pipeline.
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    #Extract feature.
    feature_maps = feature_extractor.predict(data, batch_size=32, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [11]:
#Extracting features using InceptionV3

from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input
inception_features = get_features(InceptionV3,
                                  inception_preprocessor,
                                  img_size,
                                  X)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
10222/10222 [==============================] - 1419s 139ms/sample
Feature maps shape:  (10222, 2048)


In [12]:
#Extracting features using Xception
from tensorflow.keras.applications.xception import Xception, preprocess_input
xception_preprocessor = preprocess_input
xception_features = get_features(Xception,
                                 xception_preprocessor,
                                 img_size, X)

10222/10222 [==============================] - 2708s 265ms/sample
Feature maps shape:  (10222, 2048)


In [13]:
#Extracting features using InceptionResnetV2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
inc_resnet_preprocessor = preprocess_input
inc_resnet_features = get_features(InceptionResNetV2,
                                   inc_resnet_preprocessor,
                                   img_size, X)

10222/10222 [==============================] - 3359s 329ms/sample
Feature maps shape:  (10222, 1536)


In [14]:
#concatinating features
final_features = np.concatenate([inception_features,
                                 xception_features,
                                 inc_resnet_features,], axis=-1)
print('Final feature maps shape', final_features.shape)

Final feature maps shape (10222, 5632)


In [15]:
del X

In [16]:
#Callbacks
from tensorflow.keras.callbacks import EarlyStopping
EarlyStop_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
my_callback=[EarlyStop_callback]

In [17]:
#Building Model
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(InputLayer(final_features.shape[1:]))
model.add(Dropout(0.7))
model.add(Dense(120,activation='softmax'))

In [18]:
#Compiling Model
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [19]:
#Training Model
history = model.fit(final_features,
                  y,
                  batch_size=32,
                  epochs=50,
                  validation_split=0.1,
                  callbacks=my_callback)

Train on 9199 samples, validate on 1023 samples
Epoch 1/50
9199/9199 [==============================] - 7s 725us/sample - loss: 0.8512 - acc: 0.7993 - val_loss: 0.3509 - val_acc: 0.8876
Epoch 2/50
9199/9199 [==============================] - 4s 434us/sample - loss: 0.2634 - acc: 0.9160 - val_loss: 0.3647 - val_acc: 0.9013
Epoch 3/50
9199/9199 [==============================] - 4s 407us/sample - loss: 0.2119 - acc: 0.9327 - val_loss: 0.3885 - val_acc: 0.8974
Epoch 4/50
9199/9199 [==============================] - 4s 401us/sample - loss: 0.1796 - acc: 0.9389 - val_loss: 0.3800 - val_acc: 0.8974
Epoch 5/50
9199/9199 [==============================] - 4s 397us/sample - loss: 0.1472 - acc: 0.9497 - val_loss: 0.4323 - val_acc: 0.9032
Epoch 6/50
9199/9199 [==============================] - 4s 389us/sample - loss: 0.1341 - acc: 0.9553 - val_loss: 0.4177 - val_acc: 0.9013
Epoch 7/50
9199/9199 [==============================] - 4s 385us/sample - loss: 0.1209 - acc: 0.9572 - val_loss: 0.4407 - va

In [20]:
#Converting test images to array
def images_to_array2(data_dir,df, img_size):
    images_names = df['id']
    data_size = len(images_names)
    X = np.zeros([data_size, img_size[0], img_size[1], 3], dtype=np.uint8)
    
    for i in range(data_size):
        image_name = images_names[i]
        img_dir = os.path.join(data_dir, image_name+'.jpg')
        img_pixels = load_img(img_dir, target_size=img_size)
        X[i] = img_pixels
        
    print('Ouptut Data Size: ', X.shape)
    return X

In [21]:
sample_df = pd.read_csv('C:/dev/dog-breed-identification/sample_submission.csv')

In [22]:
test_data = images_to_array2(test_dir, sample_df, img_size)

Ouptut Data Size:  (10357, 299, 299, 3)


In [23]:
#Extract test data features.
inception_features = get_features(InceptionV3, inception_preprocessor, img_size, test_data)

10357/10357 [==============================] - 1637s 158ms/sample
Feature maps shape:  (10357, 2048)


In [24]:
xception_features = get_features(Xception, xception_preprocessor, img_size, test_data)

10357/10357 [==============================] - 2889s 279ms/sample
Feature maps shape:  (10357, 2048)


In [ ]:
inc_resnet_features = get_features(InceptionResNetV2, inc_resnet_preprocessor, img_size, test_data)

 3200/10357 [========>.....................] - ETA: 53:41

In [ ]:
test_features = np.concatenate([inception_features,
                                 xception_features,
                                 inc_resnet_features],axis=-1)
print('Final feature maps shape', test_features.shape)

In [ ]:
from keras.models import load_model

model.save('dog_kind_model.h5')

In [ ]:
del test_data

In [ ]:
y_pred = model.predict(test_features, batch_size=32)

In [ ]:
for breed in dog_breeds:
    sample_df[breed] = y_pred[:,class_to_num[breed]]
sample_df.to_csv('pred.csv', index=None)

In [ ]:
sample_df.head()